In [ ]:
import pandas
import mlxtend
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

### Generate Association Rules

In [ ]:
df = pandas.read_csv("/Users/Konstantin/git/otus/sql.csv", sep=";")

In [ ]:
df_stats = df.groupby("id")['title'].apply(list)

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(df_stats).transform(df_stats)

import re
new_columns = [re.sub("-\d{4}-\d{2}","", x) for x in te.columns_]

df = pandas.DataFrame(te_ary, columns=new_columns)
frequent_itemsets = apriori(df, min_support=0.001, use_colnames=True)
# frequent_itemsets = frequent_itemsets[frequent_itemsets.itemsets.apply(len) > 1]
frequent_itemsets

In [ ]:
ar = association_rules(frequent_itemsets, min_threshold=0.05, metric="confidence")
ar = ar.loc[:,['antecedents','consequents','support','confidence','lift']].sort_values(by="lift", ascending=False)

### Generate most probable Next Course

In [ ]:
df = pandas.read_csv("/Users/Konstantin/git/otus/sql.csv", sep=";")
purchases_dataset = df.groupby("id").title.apply(list)

In [ ]:
def make_itemsets(course_list):
    if len(course_list) < 2:
        return []
    course_list_dates = [re.search("\d{4}-\d{2}",x).group(0) for x in course_list if re.search("\d{4}-\d{2}",x) != None]
    course_list_titles = [re.sub("-\d{4}-\d{2}","",x) for x in course_list]
    course_list = sorted(zip(course_list_titles, course_list_dates), key= lambda x: x[1])
    current_result = []
    for x in range(len(course_list)):
        for y in range(x+1, len(course_list)):
            current_result.append((course_list[x][0], course_list[y][0]))
    return current_result

Make all pairs of courses (prev, next) for each customer (where next > prev) and put into list

In [ ]:
result = []
for customer_purchases in purchases_dataset:
    course_pairs = make_itemsets(customer_purchases)
    if len(course_pairs) > 0:
        result = result + course_pairs

Convert lists to pandas dataframe

In [168]:
df_prob = pandas.DataFrame(result, columns=['prev','next'])

In [181]:
# Use dummy variable to compute stats
df_prob['dummy'] = 1

# Count Pairs (prev, next)
df_agg = df_prob.groupby(["prev","next"], as_index=False)['dummy'].sum()

# Count pairs with current Prev (further will be normalized by that)
df_sum = df_agg.groupby("prev", as_index=False)['dummy'].sum()

# Merge current Prev counts
df_agg = pandas.merge(left=df_agg, right=df_sum, on="prev")

# Normalize
df_agg['prob'] = round(df_agg['dummy_x'] / df_agg['dummy_y'], 2)

# Sort By Next Probability
df_agg.sort_values(by=["prev","prob"], ascending=[True, False], inplace=True)

df_agg = df_agg.rename(columns={"dummy_x":"freq"})

# Export to Disk
df_agg[['prev','next','prob', 'freq']].to_csv("/Users/Konstantin/git/otus/result.csv", sep=";", index=False)